In [ ]:
import numpy as np
import glob
import os
import cv2
import numpy as np
from google.colab.patches import cv2_imshow
import imutils

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
rawdatafolder = 'SmilesDataset/grid_data'
croppedfolder = 'SmilesDataset/shame_data'
rawdatapath = os.path.join('/content/gdrive/MyDrive',rawdatafolder)
croppedpath = os.path.join('/content/gdrive/MyDrive',croppedfolder)

In [ ]:
def split_grid(path, save_path, rotate=False):
    image  = cv2.imread(path)
    image = cv2.resize(image, (437, 437), interpolation=cv2.INTER_AREA)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (5,5), 0)
    thresh = cv2.adaptiveThreshold(blur, 255, 1, 1, 11, 2)

    contours = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    contours = imutils.grab_contours(contours)

    h, w, _ = image.shape
    min_area = (h/10.)*(w/10.) - 30**2
    max_area = (h/10.)*(w/10.) + 30**2

    i = 0
    for contour in contours:
        area = cv2.contourArea(contour)
        if area > min_area and area < max_area:
            x,y,w,h = cv2.boundingRect(contour)
            ROI = image[y:y+h, x:x+w]
            resized_roi = cv2.resize(ROI, (32, 32), interpolation=cv2.INTER_AREA)
            gray_roi = cv2.cvtColor(resized_roi, cv2.COLOR_RGB2GRAY)
            ret_roi,wb_roi = cv2.threshold(gray_roi,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
            if rotate:
                wb_roi = cv2.rotate(wb_roi, cv2.ROTATE_90_CLOCKWISE)
            imgname = os.path.splitext(os.path.split(path)[1])[0] + f"_ROI_{i}.png"
            wb_roi[wb_roi.shape[0]-2:,:] = 255
            wb_roi[:, wb_roi.shape[1]-2:] = 255
            wb_roi[:2,:] = 255
            wb_roi[:, :2] = 255
            cv2.imwrite(os.path.join(save_path,imgname), wb_roi)
            i = i + 1

In [ ]:
for filename in glob.glob(os.path.join(rawdatapath, "*_?.jpg")):
    if filename.endswith("2.jpg"):
        split_grid(filename, croppedpath, True)
    else:
        split_grid(filename, croppedpath)